In [1]:
import pandas as pd
import glob, os

# 1. Wylistuj pliki
files = glob.glob("*.csv")
print("Znalezione pliki:", files)

# 2. Wczytaj pierwszy plik bez parse_dates i obejrzyj nazwy kolumn
sample = pd.read_csv(files[0], nrows=5)
print("\nKolumny w pliku", files[0], ":\n", sample.columns.tolist())
sample.head()

Znalezione pliki: ['alerts_log.csv', 'alerty.csv', 'HDFCBANK_minute.csv', 'ICICIBANK_minute.csv', 'INFY_minute.csv', 'RELIANCE_minute.csv', 'TCS_minute.csv']

Kolumny w pliku alerts_log.csv :
 ['timestamp', 'symbol', 'close', 'volume', 'zscore', 'alert_type', 'logged_at']


,timestamp,symbol,close,volume,zscore,alert_type,logged_at
0,2015-02-02T10:20:00,INFY,534.50,813820,5.38,[⚠️ WARNING],2025-06-05T20:23:45.109701
1,2015-02-02T11:40:00,RELIANCE,215.50,731120,5.38,[⚠️ WARNING],2025-06-05T20:23:45.474486
2,2015-02-02T11:46:00,INFY,536.98,408712,5.38,[⚠️ WARNING],2025-06-05T20:23:45.499082
3,2015-02-02T12:22:00,ICICIBANK,319.00,1160519,5.36,[⚠️ WARNING],2025-06-05T20:23:45.653754
4,2015-02-02T13:12:00,TCS,1249.43,202476,5.38,[⚠️ WARNING],2025-06-05T20:23:45.842691


In [3]:
files = glob.glob("*_minute.csv")
print("Znalezione pliki:", files)

dfs = []
for fn in files:
    ticker = os.path.basename(fn).split("_minute")[0]
    
    # Wczytanie i parsowanie kolumny 'date'
    df_tmp = pd.read_csv(fn, parse_dates=["date"])
    
    # Zmiana nazwy
    df_tmp.rename(columns={"date": "DateTime"}, inplace=True)
    
    # Dodanie kolumny z tickerem
    df_tmp["Symbol"] = ticker
    
    dfs.append(df_tmp)

# 3. Scal wszystko w jeden DataFrame
df = pd.concat(dfs, ignore_index=True)

# 4. Sortowanie i reset indeksu
df.sort_values(["Symbol", "DateTime"], inplace=True)
df.reset_index(drop=True, inplace=True)

# 5. Podsumowanie
print(f"Wczytano {len(files)} plików, łącznie {df.shape[0]} wierszy")
print(df.groupby("Symbol")["DateTime"].agg(["min", "max", "count"]))
df.head()


Znalezione pliki: ['HDFCBANK_minute.csv', 'ICICIBANK_minute.csv', 'INFY_minute.csv', 'RELIANCE_minute.csv', 'TCS_minute.csv']
Wczytano 5 plików, łącznie 4517554 wierszy
                          min                 max   count
Symbol                                                   
HDFCBANK  2015-02-02 09:15:00 2024-11-08 15:29:00  903519
ICICIBANK 2015-02-02 09:15:00 2024-11-08 15:29:00  903521
INFY      2015-02-02 09:15:00 2024-11-08 15:29:00  903522
RELIANCE  2015-02-02 09:15:00 2024-11-08 15:29:00  903474
TCS       2015-02-02 09:15:00 2024-11-08 15:29:00  903518


,DateTime,open,high,low,close,volume,Symbol
0,2015-02-02 09:15:00,532.90,532.90,530.20,530.90,12719,HDFCBANK
1,2015-02-02 09:16:00,531.55,531.90,530.75,530.75,9437,HDFCBANK
2,2015-02-02 09:17:00,530.75,531.70,530.75,531.45,3500,HDFCBANK
3,2015-02-02 09:18:00,531.45,531.45,530.40,530.50,5203,HDFCBANK
4,2015-02-02 09:19:00,530.50,530.75,529.80,529.80,3386,HDFCBANK


In [5]:
# Sortowanie
df.sort_values(["Symbol","DateTime"], inplace=True)
df.reset_index(drop=True, inplace=True)

# 2. Procentowa zmiana ceny (close względem poprzedniego ticka)
df["pct_change"] = df.groupby("Symbol")["close"].pct_change() * 100
# Zastępujemy inf/-inf na NaN, potem można w razie potrzeby wyczyścić lub uzupełnić
df["pct_change"] = df["pct_change"].replace([float("inf"), float("-inf")], pd.NA)

df.dropna(subset=["pct_change"], inplace=True)

# 3. Z-score wolumenu w oknie 30-minutowym
grouped = df.groupby("Symbol")["volume"]
df["vol_mean_30"]  = grouped.rolling(window=30, min_periods=5).mean().reset_index(0,drop=True)
df["vol_std_30"]   = grouped.rolling(window=30, min_periods=5).std().reset_index(0,drop=True)
df["vol_zscore"]   = (df["volume"] - df["vol_mean_30"]) / df["vol_std_30"]

In [132]:
df.head()

,timestamp,symbol,close,volume,zscore,alert_type,logged_at
0,2015-02-02T10:20:00,INFY,534.50,813820,5.38,[⚠️ WARNING],2025-06-04T19:34:02.422790
1,2015-02-02T11:40:00,RELIANCE,215.50,731120,5.38,[⚠️ WARNING],2025-06-04T19:34:02.693533
2,2015-02-02T11:46:00,INFY,536.98,408712,5.38,[⚠️ WARNING],2025-06-04T19:34:02.712969
3,2015-02-02T12:22:00,ICICIBANK,319.00,1160519,5.36,[⚠️ WARNING],2025-06-04T19:34:02.834758
4,2015-02-02T13:12:00,TCS,1249.43,202476,5.38,[⚠️ WARNING],2025-06-04T19:34:02.998609


In [29]:

# Parametry alertów
BUY_PCT     = 2.5
SELL_PCT    = -2.5
VOL_BUY     = 2.0
VOL_SELL    = 2.0
WARN_VOL    = 5.0
WARN_PCT    = 1.2

def detect_anomalies(df):
    alerts = []

    for _, row in df.iterrows():
        pct_change = row.get("pct_change", 0)
        vol_zscore = row.get("vol_zscore", 0)
        timestamp = row.get("DateTime")
        symbol = row.get("Symbol")
        close = row.get("close")
        volume = row.get("volume")

        alert_type = None

        if pct_change > BUY_PCT and vol_zscore >= VOL_BUY:
            alert_type = "[🚀 BUY]"
        elif pct_change < SELL_PCT and vol_zscore >= VOL_SELL:
            alert_type = "[🔻 SELL]"
        elif abs(vol_zscore) >= WARN_VOL:
            alert_type = "[⚠️ WARNING]"
        elif abs(pct_change) >= WARN_PCT and abs(vol_zscore) < VOL_BUY:
            alert_type = "[⚠️ WARNING]"

        if alert_type:
            alerts.append({
                "timestamp": timestamp,
                "symbol": symbol,
                "close": close,
                "volume": volume,
                "zscore": round(vol_zscore, 2),
                "alert_type": alert_type
            })

    alert_df = pd.DataFrame(alerts)
    return alert_df


In [31]:
alert_df = detect_anomalies(df)

In [33]:
alert_df['alert_type'].value_counts()

alert_type
[⚠️ WARNING]    9669
[🔻 SELL]         322
[🚀 BUY]          277
Name: count, dtype: int64

In [123]:
# alert_df.to_csv("alerty.csv", index=False)

In [130]:
print(df.columns)

Index(['timestamp', 'symbol', 'close', 'volume', 'zscore', 'alert_type',
       'logged_at'],
      dtype='object')


In [18]:
from kafka import KafkaConsumer
from collections import deque, defaultdict
import numpy as np
from datetime import datetime

# Parametry alertów
BUY_PCT     = 2.5 # pct_change > 2,5% → kupuj
SELL_PCT    = -2.5 # pct_change < -2,5% → sprzedaj
VOL_BUY     = 2.0 # vol_zscore ≥ 2 → wolumen potwierdza ruch
VOL_SELL    = 2.0 # analogicznie dla sprzedaży
WARN_VOL    = 5.0 # wolumen bardzo wysoki → ostrzeżenie
WARN_PCT    = 1.2 # umiarkowany ruch cenowy → ostrzeżenie

# Konfiguracja konsumenta
consumer = KafkaConsumer(
    'prices',
    bootstrap_servers='localhost:9092',
    auto_offset_reset='latest',
    enable_auto_commit=True,
    value_deserializer=lambda m: json.loads(m.decode('utf-8'))
)

# Funkcja logująca alert
def log_alert(alert_type, tick, score):
    row = {
        "timestamp": tick["timestamp"],
        "symbol": tick["Symbol"],
        "close": tick["close"],
        "volume": tick["volume"],
        "zscore": round(score, 2),
        "alert_type": alert_type,
        "logged_at": datetime.utcnow().isoformat()
    }
    df = pd.DataFrame([row])
    df.to_csv("alerts_log.csv", mode='a', header=False, index=False)
    
# Bufory
buffers = defaultdict(lambda: deque(maxlen=30))

print("🛰️ Nasłuchiwanie ticków i wykrywanie anomalii...")

df = pd.read_csv("alerts_log.csv")
df.iloc[0:0].to_csv("alerts_log.csv", index=False)

for message in consumer:
    tick = message.value
    timestamp = tick["timestamp"]
    symbol = tick["Symbol"]
    close = tick["close"]
    volume = tick["volume"]
    
    buffers[symbol].append({"close": close, "volume": volume})


    # Liczenie cech przy wystarczającej liczbie danych
    if len(buffers[symbol]) >= 5:
        data = list(buffers[symbol])
        closes = [x["close"] for x in data]
        volumes = [x["volume"] for x in data]

        # pct_change – zmiana % ceny
        pct_change = ((closes[-1] - closes[-2]) / closes[-2]) * 100 if closes[-2] != 0 else 0

        # vol_zscore – z-score wolumenu
        vol_mean = np.mean(volumes)
        vol_std = np.std(volumes)
        vol_zscore = (volume - vol_mean) / vol_std if vol_std != 0 else 0

        # flagi
        flag_no_volume = int(volume == 0)
        flag_ohlc_error = int(
            tick["close"] < tick["low"] or tick["close"] > tick["high"] or
            tick["open"] < tick["low"] or tick["open"] > tick["high"]
        )

        # Reguły anomalii
        # if vol_zscore > 7 and abs(pct_change) < 0.1:
        #     alert_type = "[📊 WATCH]"
        #     print(f" {alert_type} {timestamp} | {symbol} | Ekstremalny wolumen bez zmiany ceny ({vol_zscore:.2f}σ) → Obserwuj")
        #     log_alert(alert_type, tick, round(vol_zscore, 2))
        
                # elif flag_no_volume:
        #     alert_type = "[⏸ HOLD]"
        #     print(f"{alert_type} {timestamp} | {symbol} | Brak wolumenu w godzinach handlu → Wstrzymaj się z transakcjami")
        #     log_alert(alert_type, tick, round(vol_zscore, 2))

#         elif flag_ohlc_error:
#             alert_type = "[⚠️ HOLD]"
#             print(f"{alert_type} {timestamp} | {symbol} | Błąd OHLC → Zweryfikuj dane, nie podejmuj decyzji")
#             log_alert(alert_type, tick, round(vol_zscore, 2))

#         elif abs(pct_change) > 3.0 and abs(vol_zscore) < 1.5:
#             alert_type = "[⌛ REVIEW]"
#             print(f"{alert_type} {timestamp} | {symbol} | Duży ruch ceny ({pct_change:.2f}%), wolumen ok ({vol_zscore:.2f}σ) → Poczekaj na potwierdzenie")
#             log_alert(alert_type, tick, round(vol_zscore, 2))

        if pct_change > BUY_PCT and vol_zscore >= VOL_BUY:
            alert_type = "[🚀 BUY]"
            print(f"{alert_type} {timestamp} | {symbol} | Wzrost {pct_change:.2f}% + wolumen {vol_zscore:.2f}σ → Sygnał do kupna")
            log_alert(alert_type, tick, round(vol_zscore, 2))

        elif pct_change < SELL_PCT and vol_zscore >= VOL_SELL:
            alert_type = "[🔻 SELL]"
            print(f"{alert_type} {timestamp} | {symbol} | Spadek {pct_change:.2f}% + wolumen {vol_zscore:.2f}σ → Sygnał do sprzedaży")
            log_alert(alert_type, tick, round(vol_zscore, 2))

        elif abs(vol_zscore) >= WARN_VOL:
            alert_type = "[⚠️ WARNING]"
            print(f"{alert_type} {timestamp} | {symbol} | Wolumen: {vol_zscore: .2f}σ → Bądź czujny - Bardzo duży wolumen ")
            log_alert(alert_type, tick, round(vol_zscore, 2))
            
        elif abs(pct_change) >= WARN_PCT and abs(vol_zscore) < VOL_BUY:
            alert_type = "[⚠️ WARNING]"
            print(f"{alert_type} {timestamp} | {symbol} | Ruch ceny: {pct_change: .2f}% (ₓ), wolumen: {vol_zscore: .2f}σ (niski) → Uwaga - możliwa manipulacja, wzrosty bez wolumenu")
            log_alert(alert_type, tick, round(vol_zscore, 2))
            

🛰️ Nasłuchiwanie ticków i wykrywanie anomalii...
[⚠️ WARNING] 2015-02-02T10:20:00 | INFY | Wolumen:  5.38σ → Bądź czujny - Bardzo duży wolumen 
[⚠️ WARNING] 2015-02-02T11:40:00 | RELIANCE | Wolumen:  5.38σ → Bądź czujny - Bardzo duży wolumen 
[⚠️ WARNING] 2015-02-02T11:46:00 | INFY | Wolumen:  5.38σ → Bądź czujny - Bardzo duży wolumen 
[⚠️ WARNING] 2015-02-02T12:22:00 | ICICIBANK | Wolumen:  5.36σ → Bądź czujny - Bardzo duży wolumen 
[⚠️ WARNING] 2015-02-02T13:12:00 | TCS | Wolumen:  5.38σ → Bądź czujny - Bardzo duży wolumen 
[⚠️ WARNING] 2015-02-02T13:30:00 | ICICIBANK | Wolumen:  5.01σ → Bądź czujny - Bardzo duży wolumen 
[⚠️ WARNING] 2015-02-02T13:37:00 | INFY | Wolumen:  5.21σ → Bądź czujny - Bardzo duży wolumen 
[⚠️ WARNING] 2015-02-02T14:15:00 | INFY | Wolumen:  5.38σ → Bądź czujny - Bardzo duży wolumen 
[⚠️ WARNING] 2015-02-02T14:21:00 | RELIANCE | Wolumen:  5.32σ → Bądź czujny - Bardzo duży wolumen 
[⚠️ WARNING] 2015-02-02T14:31:00 | TCS | Wolumen:  5.11σ → Bądź czujny - Bardzo

KeyboardInterrupt: 

In [73]:
alerty = pd.read_csv("alerts_log.csv")

In [75]:
alerty.head(20)

,timestamp,symbol,close,volume,zscore,alert_type,logged_at
0,2015-06-12T09:15:00,INFY,1003.28,17882,-1.97,[⚠️ WARNING],2025-06-04T18:39:19.915901
1,2015-06-15T09:15:00,INFY,484.60,93524,4.63,[🔻 SELL],2025-06-04T18:39:21.177235
2,2015-06-29T09:15:01,ICICIBANK,273.64,202654,4.45,[🔻 SELL],2025-06-04T18:39:33.762685
3,2015-06-29T09:15:01,RELIANCE,229.75,319030,4.46,[🔻 SELL],2025-06-04T18:39:33.764106
4,2015-07-21T09:15:01,INFY,539.83,1508054,5.38,[🚀 BUY],2025-06-04T18:39:53.969255
5,2015-07-27T09:15:01,ICICIBANK,267.00,382104,3.78,[🔻 SELL],2025-06-04T18:39:59.003929
6,2015-08-03T09:15:00,ICICIBANK,281.05,717154,1.17,[⚠️ WARNING],2025-06-04T18:40:05.303214
7,2015-08-12T09:15:00,ICICIBANK,271.09,180110,0.99,[⚠️ WARNING],2025-06-04T18:40:14.153679
8,2015-08-24T09:15:01,RELIANCE,207.20,668024,5.13,[🔻 SELL],2025-06-04T18:40:24.275138
9,2015-08-24T09:15:01,HDFCBANK,519.10,33495,1.75,[⚠️ WARNING],2025-06-04T18:40:24.277281
